## Background

**Pump state to reproduce**
1. version 16.50
2. Medium Titration set to on (default is off)
3. autherization level set to medium (default for Nurses)

**Actions to reproduce**
1. change rate

**Results**

prompt for authorization PIN -> fail

**Expected results**

Prompt for rate change -> rate change

## Analysis

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

### Importing raw data

Set dat paths

In [3]:
data_path = os.getenv('SHARED_DATA_PATH')  # env variable for data storage
logevents_path = os.path.join(data_path, r'titration_of_rate\logevents.csv')
logevents_233_path = os.path.join(data_path, r"titration_of_rate\233logevents.csv")
# these files are used to construct the total treatment list
us_excel_data = os.path.join(data_path, r"titration_of_rate\us-treatments.xlsx")
eu_excel_data = os.path.join(data_path, r"titration_of_rate\eu-treatments.xlsx")
# this path points to the total treatments csv
treatments_path = os.path.join(data_path, r"titration_of_rate\treatments.csv")
accountid_map_path = os.path.join(data_path, r"static\account_map.pkl")

Get a list of all pump serials that have listed event 233 (Medium titration) in history. Use this list to narrow down all subsequent data sets

logevents_233 is a table of all code 233 events in the elastic DB (only pumps with connect). This table maps all pumps which have the ```Medium Titration``` option set to **On**

In [4]:
usecols = ['code','dateTime', 'index', 'serialNumber', 'values']  # values - string 'On', 
df_c233 = pd.read_csv(logevents_233_path, usecols=usecols)
df_c233['dateTime'] = pd.to_datetime(df_c233['dateTime'], dayfirst=True)
df_c233['index'] = df_c233['index'].str.replace(',', '').astype(int)  # elastic integer strings may contain commas
df_c233.sort_values(by=['serialNumber', 'dateTime'])
# Not shown: for this set all values are ON -> keep only the first occurrence for each serial
df_c233 = df_c233.sort_values('dateTime').groupby('serialNumber', as_index=False).first()
# get a list of all serials with Medium Titration set to On event in history
serials = list([int(x) for x in df_c233['serialNumber'].unique()])
print(f"A total of {len(serials)} serials have Medium Titration set to 'On'")
print(f"Serial numbers:\n {serials}")

A total of 33 serials have Medium Titration set to 'On'
Serial numbers:
 [300234268, 300235397, 300244687, 300273026, 300273182, 300292679, 300316813, 300316851, 300327020, 300332207, 300339933, 300353356, 300359779, 300386836, 300392585, 300405233, 300405239, 300411832, 300413333, 300413343, 300419458, 300419541, 300426241, 300428195, 300428226, 300433481, 360217960, 370022064, 370022098, 370031546, 380019718, 700035793, 700056618]


**Only 33 serials in total are recorded (on elastic) as being potentially effected by this bug**

execute next block once to generate the full treatments CSV. Concatenates European and USA data, renames column serialNumber to serial_number and sets the correct data types.
After running once the DataFrame can be loaded from ```treatments_path```.

#### Import raw treatments

In [7]:
# Helper function required to deal with negative values for 'duration' (bug in raw data) 
def safe_timedelta(x):
    try:
        return pd.to_timedelta(x)
    except ValueError:
        return pd.NaT

# load excel data and merge
us_treatments = pd.read_excel(us_excel_data)
us_treatments['region'] = 'US'
eu_treatments = pd.read_excel(eu_excel_data)
eu_treatments['region'] = 'EU'
# join European and USA data
treatments = pd.concat([us_treatments, eu_treatments])
# treatments.rename(columns={'serialNumber': 'serial_number'}, inplace=True)
treatments['serialNumber'] = pd.to_numeric(treatments['serialNumber'], errors='coerce').astype('Int64')
treatments['startTime'] = pd.to_datetime(treatments['startTime'], format='mixed')
treatments['endTime'] = pd.to_datetime(treatments['endTime'], format='mixed')
treatments['startTime'] = pd.to_datetime(treatments['startTime'], format='mixed')
treatments['endTime'] = pd.to_datetime(treatments['endTime'], format='mixed')
treatments['softwareUpdateTime'] = pd.to_datetime(treatments['endTime'], format='mixed')
treatments['duration'] = treatments['duration'].apply(safe_timedelta)
# Narrow down treatments to serials with Medium Titration set to On
treatments = treatments[treatments['serialNumber'].isin(serials)]
# Save to local CSV to save processing time in subsequent code execution
treatments.to_csv(treatments_path, index=False)
print(f"Treatments saved to: {treatments_path} as csv")

Treatments saved to: C:\Users\aviad.baram\Data\titration_of_rate\treatments.csv as csv


In [5]:
usecols = ['serialNumber', 'startTime', 'endTime' ,'duration', 'treatmentStatus', 'deliverMode', 'region', 'vi', 'vtbiExpected']
treatments = pd.read_csv(treatments_path, usecols=usecols)
treatment_serials = treatments['serialNumber'].unique()
print(f"Total treatments: {len(treatments.index)}")
print(f"total available serials in treatments: {len(treatment_serials)}")
print(f"Serials: {treatment_serials}")
print(f"{len(serials) - len(treatment_serials)} serials are not in the treatment data")
m_serials = [sr for sr in serials if sr not in treatment_serials]
print(f"Missing serials:\n{m_serials}")

Total treatments: 1407
total available serials in treatments: 28
Serials: [300419458 300353356 300339933 300359779 370022098 700035793 370031546
 370022064 300413333 300392585 300316851 300235397 300327020 300292679
 300413343 300234268 300419541 300428195 300405233 300405239 360217960
 300433481 300428226 300386836 300273026 380019718 300411832 300426241]
5 serials are not in the treatment data
Missing serials:
[300244687, 300273182, 300316813, 300332207, 700056618]


Five serials that recorded a 233 event (i.e. Medium Titration set to On) are missing from data ⇨ not associated with customer id ⇨ assumed these are demonstration pumps

### Estimation of all affected pumps: Elastic represents 2.5% of the data ⇨ estimated total affected fleet: 1100 pumps

In [6]:
treatments.head()

,serialNumber,treatmentStatus,startTime,endTime,duration,vi,vtbiExpected,deliverMode,region
0,300419458,Completed,2025-05-13 08:03:57+00:00,2025-05-13 22:22:12+00:00,NaN,128.0,128.0,Continuous,US
1,300419458,Completed,2025-05-13 22:22:28+00:00,2025-05-16 07:32:31+00:00,2 days 09:10:03,128.0,128.0,Continuous,US
2,300419458,Partial,2025-05-16 08:53:00+00:00,2025-05-18 14:39:38+00:00,2 days 05:46:38,88.0,1032.0,Continuous,US
3,300419458,Completed,2025-05-18 14:41:06+00:00,2025-05-23 16:21:07+00:00,5 days 01:40:01,682.0,650.0,Continuous,US
4,300419458,Ongoing,2025-05-25 03:34:23+00:00,NaN,NaN,22.0,650.0,Continuous,US


### Import raw events

In [19]:
usecols = ['code', 'dateTime', 'description', 'serialNumber', 'index', 'values']
events = pd.read_csv(logevents_path, usecols=usecols)
# for debugging and testing - after : remove next line
# events = pd.read_csv(test_path, usecols=usecols)
events['dateTime'] = pd.to_datetime(events['dateTime'], format='mixed')
events['dateTime'] = events['dateTime'].dt.tz_localize('UTC')
events.sort_values(by=['serialNumber', 'dateTime'])
events = events[events['serialNumber'].isin(serials)]
print(f"Total events: {len(events.index)}")
print(f"total available serials in events: {len(events['serialNumber'].unique())}")
print(f"Serials: {events['serialNumber'].unique()}")

Total events: 334870
total available serials in events: 33
Serials: [300428195 300405239 300413333 300235397 300327020 300392585 300316851
 300413343 370022098 300359779 300332207 360217960 300339933 370031546
 370022064 300316813 300419541 300244687 300405233 300433481 300426241
 300428226 300292679 300273026 380019718 300411832 300386836 300234268
 300273182 300353356 300419458 700056618 700035793]


In [20]:
# set the codes required from the exhaustive list of 
codes = [233, # authorization level set to On
    376, # rate was changed - Normal: expected in 16.20
    387, # Invalid password attempt - Bug expected in 16.50 when attempting Medium Titration
     58, # authorization level changed value: Low, Medium,High,Technician
     51, # delivery mode Continuous,Intermittent,TPN,PCA,Secondary,Multi-step,Epidural PCA,Epidural Intermittent,Epidural. sanity check against treatments
     57, # power on. Resets Technician modes to Medium
     73, # new infusion started
     222, # new treatment wizard initiated - indication of started
     74, # pump started
     123, # SW version stored in value
     130  # infusion ended with info
    ]
events = events[events['code'].isin(codes)]
print("After selecting codes:")
print(f"Total events: {len(events.index)}")

After selecting codes:
Total events: 61199


This function is required to strip array values of the literal array characters. In elastice the arrays ['16', 'Technician'] is stored as the string "'['16', 'Tecnician']'"

In [22]:
def clean_auth_value(val):
    # Extracts the word inside the brackets and quotes, e.g., "['High']" -> 'High'
    match = re.match(r"\['(.+)'\]", str(val))
    if match:
        return match.group(1)
    return val

This function iterates over the events in their chrnological order. It maintains an updated state of the pump and assigns each event a series of attributes representing: state of ```Medium Titration``` setting,  Authorization level, selected infusion mode, software version and a state change flag. The state change flag indicates that the state of the pump changed following this event. 

In [23]:
def set_event_status(events_df):
    """
    Iterates over all serials in events_df. For each event in a serial set:
    Sets a status flag 'med_tit'. The flag starts at 'Off', is set to 'On' if code 233 with value 'On' is encountered, and set back to 'Off' if code 233 with value 'Off' is encountered.
    Sets a value flag for 'level'. The level flag changes on event code 58 and persists from then on excluding Technician level which also resets on pump start
    Sets a value for 'mode'. The mode flag 
    
    """
    events_df = events_df.copy()
    for serial in events_df['serialNumber'].unique():
        serial_mask = events_df['serialNumber'] == serial
        serial_events = events_df[serial_mask].sort_values('dateTime')
        med_tit = []
        levels = []
        modes = []
        version = []
        status_change_flag = []
        current_value = {'med_tit': False, 'mode':'', 'level':'Medium', 'version':''}  # med_titration is Off by default
        for idx, row in serial_events.iterrows():
            status_change_f = False
            if row['code'] == 233:  # Medium Titration setting change event. Values are 'On' and 'Off'
                if clean_auth_value(row['values']).lower() == 'on':
                    med_t = True
                else:
                    med_t = False
                if med_t != current_value['med_tit']:
                    current_value['med_tit'] = med_t
                    status_change_f = True
            if row['code'] == 58:  # Authorization level change event. 
                lvl = clean_auth_value(row['values'])
                if lvl != current_value['level']:
                    current_value['level'] = lvl
                    status_change_f = True
            if row['code'] == 57:  # system start event. Resets level Technician to Medium
                if current_value['level'] == 'Technician':
                    current_value['level'] = 'Medium'
                    status_change_f = True
            if row['code'] == 51:  # Infusion mode set event
                md = clean_auth_value(row['values'])
                if md != current_value['mode']:
                    current_value['mode'] = md
                    status_change_f = True
            if row['code'] == 123:  # version update event
                try:
                    vr = eval(row['values'])
                except:
                    vr = row['values'].split(',')
                    vr = [x.strip() for x in vr]
                vr = vr[0] + '.' + vr[1]
                if vr != current_value['version']:
                    current_value['version'] = vr
                    status_change_f = True
            levels.append(current_value['level'])
            med_tit.append(current_value['med_tit'])
            modes.append(current_value['mode'])
            version.append(current_value['version'])
            status_change_flag.append(status_change_f)
            # Assign lists to the DataFrame for this serial
        events_df.loc[serial_events.index, 'med_tit'] = med_tit
        events_df.loc[serial_events.index, 'level'] = levels
        events_df.loc[serial_events.index, 'mode'] = modes
        events_df.loc[serial_events.index, 'version'] = version
        events_df.loc[serial_events.index, 'status_change_flag'] = status_change_flag
    return events_df

# mark med titration state and level
events = set_event_status(events)


C:\Users\aviad.baram\AppData\Local\Temp\ipykernel_62904\2813576691.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, T

Assign events to treatments

In [ ]:
# count the total number of Medium level 376 type events
def assign_treatment_label(row, treatments_df, available_serials):
    serial = row['serialNumber']
    if serial not in available_serials:
        return 'serial_missing'
    matches = treatments_df[
        (treatments_df['serialNumber'] == serial) &
        (treatments_df['startTime'] <= row['dateTime']) &
        (treatments_df['endTime'] >= row['dateTime'])
    ]
    if not matches.empty:
        return matches.index[0]  # or use another unique treatment identifier
    else:
        return 'out_of_treatment'

# Precompute available serials for efficiency
available_serials = set(treatments['serialNumber'].unique())

# Assign treatment label to each event
events['treatment_assignment'] = events.apply(
    lambda row: assign_treatment_label(row, treatments, available_serials), axis=1
)

version_info = treatments.loc[treatments['serialNumber'].isin(available_serials)]
# a total of 341 treatments are identified
def assign_version_flag(row):
    # If not associated with a treatment, assign 'NA'
    if pd.isna(row['treatment_assignment']):
        return 'NA'
    # If last_software_update is after the event, assign 'ambigious'
    if pd.isna(row['last_software_update']):
        return 'ambigious'
    if row['last_software_update'] > row['dateTime']:
        return 'ambagious'
    # Otherwise, assign the current_software_version
    return row['current_software_version']

events['version_flag'] = events.apply(assign_version_flag, axis=1)

calculate probabilities

In [30]:
#count_med_titration
count_rate_change_16_20_idx  = events.loc[(events['version'] == '16.20')  &
                                      (events['code'] == 376) &
                                      (events['level'] == 'Medium') &
                                      (events['med_tit'])].index
count_rate_change_16_50_idx  = events.loc[(events['version'] == '16.50')  &
                                      (events['code'] == 376) &
                                      (events['level'] == 'Medium') &
                                      (events['med_tit'])].index
count_16_20_treatments = len(events.loc[(events['version'].str.startswith('16.2')) &
                                    events['code'] == 73].index)
count_16_50_treatments = len(events.loc[(events['version'].str.startswith('16.5')) &
                                    events['code'] == 73].index)
print(f"Total events of rate change in 16.20: {len(count_rate_change_16_20_idx)}")
try:
    prob = len(count_rate_change_16_20_idx)/ count_16_20_treatments
except ZeroDivisionError:
    prob = 'NaN'    
print(f"Probability of rate change in 16.20: {prob}")
print(f"Total events of rate change in 16.50: {len(count_rate_change_16_50_idx)} ")
try:
    prob = len(count_rate_change_16_50_idx)/ count_16_50_treatments
except ZeroDivisionError:
    prob = 'NaN'
print(f"Probability of rate change in 16.50: {prob}")
count_auth_fail_16_20_idx  = events.loc[(events['version'] == '16.20')  &
                                      (events['code'] == 387) &
                                      (events['level'] == 'Medium') & # (Required High entered Medium)
                                      (events['med_tit'])].index
count_auth_fail_16_50_idx  = events.loc[(events['version'] == '16.50')  &
                                      (events['code'] == 387) &
                                      (events['level'] == 'Medium') &
                                      (events['med_tit'])].index
count_16_50_treatments = events.loc[(events['version'].str.startswith('16.5')) &
                                    events['code'] == 73].count()
print(f"Total events of authorization failure in 16.20: {len(count_auth_fail_16_20_idx)} ")
try:
    prob = len(count_auth_fail_16_20_idx)/ count_16_20_treatments
except ZeroDivisionError:
    prob = 'NaN'    
print(f"Probability of authorization failure change in 16.20: {prob}")

print(f"Total events of authorization failure in 16.50: {len(count_auth_fail_16_50_idx)} ")
try:
    prob = len(count_auth_fail_16_50_idx)/ count_16_50_treatments
except ZeroDivisionError:
    prob = 'NaN'
# print(prob)                   
# print(f"Probability of authorization failure change in 16.50: {prob}")

Total events of rate change in 16.20: 2
Probability of rate change in 16.20: NaN
Total events of rate change in 16.50: 0 
Probability of rate change in 16.50: NaN
Total events of authorization failure in 16.20: 147 
Probability of authorization failure change in 16.20: NaN
Total events of authorization failure in 16.50: 35 


In [85]:
count_auth_fail_16_50_idx

Index([ 80091,  80115,  81390,  82255,  83961,  84795,  84819,  85065,  89069,
        93339,  99385, 104879, 115526, 115545, 138067, 140211, 145747, 150131,
       154243, 154787, 155555, 156323, 157395, 314306, 314307, 314313, 314345,
       314346, 314422, 314424, 314427, 314449, 314450, 314622, 314623],
      dtype='int64')